In [ ]:
import os
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List
from langchain.schema import Document
from dotenv import load_dotenv
from langchain_community.retrievers import ArxivRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama


# Load API key
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Initialize LLM and embeddings
llm = Ollama(model="llama3", temperature=0)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Corrected retriever initialization
retriever = ArxivRetriever(
    load_max_docs=2,
    get_full_documents=True,
)

# Define the state structure
class AgentState(TypedDict):
    search: str
    documents: List[Document]
    keywords: List[str]
    answer: str

# Example documents
sample_texts = [
    "A container is a lightweight, standalone, and executable unit that includes:",
    "Containers share the host OS kernel, which makes them faster and more resource efficient compared to Virtual Machine.",
    "A Docker Image is a read-only template used to create containers.",
    "A hypervisor ecosystem refers to the complete set of tools, technologies, and services that work with a hypervisor to manage, secure, and optimize virtualized environments."
]

documents = [Document(page_content=text) for text in sample_texts]

# Build FAISS vectorstore
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(k=3)

# Node 1
def need_line(state: AgentState):
    search = state.get("search", "")
    keywords = state.get("keywords", [])
    if not keywords:
        return {**state, "answer": "", "search": search}
    answer_found = any(keyword.lower() in search.lower() for keyword in keywords)
    return {**state, "answer": str(answer_found)}

# Node 2
def check_search(state: AgentState):
    search = state.get("search", "")
    documents = retriever.invoke(search)
    return {**state, "documents": documents}

# Node 3
def answer_doc(state: AgentState):
    search = state.get("search", "")
    documents = state.get("documents", [])
    if documents:
        context = "\n\n".join([doc.page_content for doc in documents])
        prompt = f"Based on the context below, answer the question.\nContext:\n{context}\nQuestion: {search}\nAnswer:"
    else:
        prompt = f"Answer the question based on your knowledge: {search}"

    response = llm.invoke(prompt)
    return {**state, "answer": response}

# Conditional branching
def found_search(state: AgentState):
    if state.get("answer") == "True":
        return "answer_doc"
    else:
        return "search"

# Workflow
graph = StateGraph(AgentState)

graph.add_node("need_line", need_line)
graph.add_node("check_search", check_search)
graph.add_node("answer_doc", answer_doc)

graph.add_edge(START, "need_line")

graph.add_conditional_edges(
    "need_line",
    found_search,
    {
        "answer_doc": "answer_doc",
        "search": "check_search",
    },
)

graph.add_edge("check_search", "answer_doc")
graph.add_edge("answer_doc", END)

chain = graph.compile()

# Function to ask questions
def ask_question(question: str):
    initial_state = {
        "search": question,
        "documents": [],
        "answer": "",
        "keywords": ["docker", "container", "image"],
    }
    return chain.invoke(initial_state)

# Run test
question1 = "What is Docker image?"
result1 = ask_question(question1)
print(result1)




{'search': 'What is Docker image?', 'documents': [], 'keywords': ['docker', 'container', 'image'], 'answer': "A Docker image is a lightweight, standalone package that contains an application and its dependencies. It's essentially a snapshot of a Linux environment that includes the operating system, libraries, frameworks, and other components necessary to run a specific application.\n\nIn simpler terms, a Docker image is like a virtual container that holds everything needed to run a particular program or service. When you create a Docker image, you're essentially creating a self-contained environment that can be used to deploy your application in various environments, such as development, testing, staging, and production.\n\nDocker images are built using a combination of operating system layers, which allows for efficient storage and retrieval of the image. Each layer is a separate file on disk, making it easy to manage and update individual components without affecting the entire image